# Diorama AMF inversion

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cPickle as pickle
import os

from fatiando import utils
from fatiando.gravmag import polyprism
from fatiando.mesher import PolygonalPrism
from fatiando.vis import mpl, myv
from matplotlib import colors, ticker, cm
from IPython.display import Image as img
from matplotlib.mlab import normpdf

/home/leonardo/anaconda2/lib/python2.7/site-packages/fatiando/vis/mpl.py:76: UserWarning: This module will be removed in v0.6. We recommend the use of matplotlib.pyplot module directly. Some of the fatiando specific functions will remain.
  "specific functions will remain.")
/home/leonardo/anaconda2/lib/python2.7/site-packages/fatiando/vis/myv.py:51: UserWarning: This module will be removed in v0.7.
  warnings.warn("This module will be removed in v0.7.")


### Auxiliary functions

In [2]:
import sys
sys.path.insert(0, '../../code')

import mag_polyprism_functions as mfun
import mag_sphere_functions as msf

# Input

### Importing Diorama AMF data

In [3]:
mypath = 'diorama_eqlayer/direction_inc-59_dec-114_h200m/'

In [4]:
with open(mypath+'diorama_eqlayer_depth200m.pickle') as w:
        di = pickle.load(w)

### Parameters for inversion

In [5]:
# output of inversion
diorama_inversion = dict()

In [6]:
diorama_inversion['xp'] = di['x']
diorama_inversion['yp'] = di['y']
diorama_inversion['zp'] = di['z']
diorama_inversion['observed_data'] = di['amf']

In [7]:
#initial estimate
M = 20 # number of vertices per prism
L = 10 # number of prisms
P = L*(M+2) + 1 # number of parameters

# main field
inc = -19.5
dec = -18.5

incs = -71.41751
decs = -23.39541

z0 = 350.
dz = 450.
r = 1000.
props = {'magnetization': utils.ang2vec(
        15., inc, dec)}
N = di['x'].size

#m0 = np.zeros(P-1)
#for i in range (1, L+1, 1):
    #m0[i*(M+2):i*(M+2)+M] = 2000. + (i-10)*200. - (i+5)*200.
    #m0[i*(M+2)+M:i*(M+2)+M+2] = np.array([8212500., 478200.])
rin = np.zeros(M) + r
m0 = np.hstack((rin, np.array([8212500., 478200.])))
m0 = np.resize(m0, P - 1) # inicial parameters vector
m0 = np.hstack((m0, dz))
model0 = mfun.param2polyprism(m0, M, L, z0, props) # list of classes of prisms

# predict data
d0 = np.sqrt(polyprism.bx(di['x'], di['y'], di['z'], model0)**2. +
            polyprism.by(di['x'], di['y'], di['z'], model0)**2. +
            polyprism.bz(di['x'], di['y'], di['z'], model0)**2.)

# limits for parameters in meters
rmin = 50.
rmax = 3000.
x0min = 8211000.
x0max = 8214000.
y0min = 477000.
y0max = 479000.
dzmin = 100.
dzmax = 800.

mmin, mmax = mfun.build_range_param(M, L, rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax)

itmax = 30
itmax_marq = 15
epsilon = 1.0e-5     # stop criterion

# increment for derivatives
deltax = 0.01*np.max(x0max)
deltay = 0.01*np.max(y0max)
deltar = 0.01*np.max(rmax)
deltaz = 0.01*np.max(dzmax)
delta = np.array([deltax, deltay, deltar, deltaz])
m_out = np.zeros(M + 2)  # outcropping body parameters

In [8]:
diorama_inversion['inc_dec'] = [incs, decs]
diorama_inversion['z0'] = z0
diorama_inversion['initial_dz'] = dz
diorama_inversion['intial_r'] = r
diorama_inversion['initial_estimate'] = model0
diorama_inversion['initial_data'] = d0
diorama_inversion['limits'] = [rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax]
diorama_inversion['tol'] = epsilon
diorama_inversion['main_field'] = [inc, dec]

In [9]:
mypath = 'diorama_amf_inversion/diorama_r%d_int%d_z0%d_dz%d' % (rin[0], np.linalg.norm(props['magnetization']), z0, dz)
if not os.path.isdir(mypath):
   os.makedirs(mypath)

### Regularization parameters

In [10]:
#lamb = th*0.01 # Marquadt's parameter
lamb = 10.0
dlamb = 10.      # step for Marquadt's parameter

a1 = 0.01   # adjacent radial distances within each prism
a2 = 0.01 # vertically adjacent radial distances
a3 = 0.     # outcropping cross-section
a4 = 0.     # outcropping origin
a5 = 0.01     # vertically adjacent origins
a6 = 0.00001   # zero order Tikhonov on adjacent radial distances
a7 = 0.00001     # zero order Tikhonov on thickness of each prism

In [11]:
alpha = np.array([a1, a2, a3, a4, a5, a6, a7])

In [12]:
diorama_inversion['regularization'] = alpha

### Inversion

In [13]:
d_fit, m_est, model_est, phi_list = mfun.levmarq_amf(di['x'], di['y'], di['z'], m0, M, L, delta, itmax,
                                                    itmax_marq, lamb, dlamb, epsilon,
                                                    mmin, mmax, m_out, di['amf'],
                                                    props, alpha, z0, dz)

it:  0   it_marq:  0   lambda: 1e+01   misfit: 1.6566e+06
it:  1   it_marq:  0   lambda: 1e+00   misfit: 1.7398e+06
it:  1   it_marq:  1   lambda: 1e+01   misfit: 1.6044e+06
it:  2   it_marq:  0   lambda: 1e+00   misfit: 1.6943e+06
it:  2   it_marq:  1   lambda: 1e+01   misfit: 1.6110e+06
it:  2   it_marq:  2   lambda: 1e+02   misfit: 1.6070e+06
it:  2   it_marq:  3   lambda: 1e+03   misfit: 1.6047e+06
it:  2   it_marq:  4   lambda: 1e+04   misfit: 1.6045e+06
it:  2   it_marq:  5   lambda: 1e+05   misfit: 1.6044e+06
it:  2   it_marq:  6   lambda: 1e+06   misfit: 1.6044e+06
it:  2   it_marq:  7   lambda: 1e+07   misfit: 1.6044e+06
it:  2   it_marq:  8   lambda: 1e+08   misfit: 1.6044e+06
it:  2   it_marq:  9   lambda: 1e+09   misfit: 1.6044e+06
it:  2   it_marq: 10   lambda: 1e+10   misfit: 1.6044e+06
it:  2   it_marq: 11   lambda: 1e+11   misfit: 1.6044e+06
it:  2   it_marq: 12   lambda: 1e+12   misfit: 1.6044e+06
it:  2   it_marq: 13   lambda: 1e+13   misfit: 1.6044e+06
it:  2   it_ma

In [14]:
diorama_inversion['residual'] = di['amf'] - d_fit
diorama_inversion['residual_mean'] = np.mean(diorama_inversion['residual'])
diorama_inversion['residual_std'] = np.std(diorama_inversion['residual'])
diorama_inversion['residual_norm'] = (diorama_inversion['residual'] - diorama_inversion['residual_mean'])/diorama_inversion['residual_std'] # residue normalization

In [15]:
diorama_inversion['data_fit'] = d_fit
diorama_inversion['estimate'] = m_est
diorama_inversion['prisms'] = model_est
diorama_inversion['objective'] = phi_list

In [16]:
file_name = mypath+'/diorama_inversion_r%d_int%d_z0%d_dz%d.pickle' % (rin[0], np.linalg.norm(props['magnetization']), z0, dz)
with open(file_name, 'w') as f:
    pickle.dump(diorama_inversion, f)